In [2]:
import statsmodels.api as sm 
import pandas as pd

In [3]:
df_rolled = pd.read_parquet("../data/processed/df_preprocessed.parquet")
df_rolled.shape

(125776, 29)

In [4]:
df_rolled = df_rolled.loc[df_rolled['No_Total_Sales_Flag']==0]
print(df_rolled.shape)

(70475, 29)


In [5]:
model_levels = ["PPG","Retailer_Name"] 


In [23]:
var_dep = 'Total_Volume'
vars_ind = ['Avg_Price_Per_KG','Distribution_wtd','Distribution_numeric']
vars_all = [var_dep] + vars_ind
print(vars_all)

df_rolled[vars_all].isnull().sum()

['Total_Volume', 'Avg_Price_Per_KG', 'Distribution_wtd', 'Distribution_numeric']


Total_Volume              0
Avg_Price_Per_KG          0
Distribution_wtd        270
Distribution_numeric    371
dtype: int64

In [22]:
df_rolled['Distribution_numeric'] = df_rolled['Distribution_numeric'].fillna(df_rolled.groupby(model_levels)['Distribution_numeric'].transform('mean'))
df_rolled['Distribution_wtd'] = df_rolled['Distribution_wtd'].fillna(df_rolled.groupby(model_levels)['Distribution_wtd'].transform('mean'))

In [25]:
df_rolled = df_rolled.loc[~df_rolled['Distribution_numeric'].isna()]
df_rolled = df_rolled.loc[~df_rolled['Distribution_wtd'].isna()]
df_rolled.shape

(70104, 29)

In [41]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.rsquared

In [42]:
df_rolled.groupby(model_levels,as_index=False).apply(regress, var_dep, vars_ind)

,PPG,Retailer_Name,None
0,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer A,0.820305
1,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer B,0.802672
2,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer C,0.643769
3,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer A,0.862448
4,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer B,0.832114
...,...,...,...
527,BRAND O SUBBRAND O.6 ADULT BAK RUND 60_G,Retailer C,0.423185
528,BRAND O SUBBRAND O.6 ADULT BAK ZALM 60_G,Retailer A,0.828068
529,BRAND O SUBBRAND O.6 ADULT BAK ZALM 60_G,Retailer B,0.654889
530,BRAND O SUBBRAND O.6 ADULT BAK ZALM 60_G,Retailer C,0.615018
